# EP 3.2 -  Visão e Processamento de Imagens

###Integrantes do grupo:

-> Ciro Akiyoshi Higashi - nusp:10736858

-> Lucas Giannella de Oliveira - nusp:10336021

-> Link do Git: https://github.com/Cirokun/MAC0417-MAC5768

-> Link para o Google Drive: https://drive.google.com/drive/folders/18tiGKyk2206Kd_v0qlBazA4Hcn5zeODi?usp=sharing

## Importação de bibliotecas

In [ ]:
import os
import re
import sys
import math
import numpy as np
from sklearn import neural_network,naive_bayes
from skimage import io, color, exposure, filters, transform,measure,morphology
from sklearn import decomposition,model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC

#Conexão com o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Mudança para o diretorio de trabalho

In [ ]:
os.chdir('/content/drive/MyDrive/MAC5768/EPs')

## Função para recolher os dados
-> A função a seguir recebe dois vetores (Entrada e Respostas) e um diretorio

-> Do diretorio lê todas as imagens e as coloca no vetor de entrada ($\textbf{X}$)

-> A partir dos metadados no nome da imagem cria-se o vetor das respostas ($\textbf{Y}$)

Obs: Durantes os testes iniciais colocar todas as imagens nos vetores estava extrapolando a RAM maxima do ambiente de execução, portanto optamos por utilizar versões redimensionadas (reduzidas) das imagens segmentas. 

Esta reduçao foi de 1/8 diminuindo as imagens de 4000x2248 para 500x281.
As imagens redimensionadas foram inspecionadas manualmente para verificar que não havia ocorrido perda de informações significativas

In [ ]:
def loadXY(pathMan,X,Y):
    for filename in os.listdir(pathMan):
        img = io.imread(os.path.join(pathMan,filename), as_gray=True)
        classe=(filename.split("_")[0])[:-1]
        if img is not None:
            X = np.append(X, np.array(img).reshape((1,X.shape[1])), axis=0)
            if (classe == "Caneca") :
                Y=np.append(Y, np.array([[0]]), axis=0)
            elif (classe == "Chave") :
                Y=np.append(Y, np.array([[1]]), axis=0)
            elif (classe == "Escova") :
                Y=np.append(Y, np.array([[2]]), axis=0)
            elif (classe == "Dado") :
                Y=np.append(Y, np.array([[3]]), axis=0)
            elif (classe == "Meia") :
                Y=np.append(Y, np.array([[4]]), axis=0)
            elif (classe == "Fidget") :
                Y=np.append(Y, np.array([[5]]), axis=0)
            elif (classe == "Laranja") :
                Y=np.append(Y, np.array([[6]]), axis=0)
            elif (classe == "Garfo") :
                Y=np.append(Y, np.array([[7]]), axis=0)
            elif (classe == "Moeda") :
                Y=np.append(Y, np.array([[8]]), axis=0)
            elif (classe == "Prato") :
                Y=np.append(Y, np.array([[9]]), axis=0)
    return X,Y

Criação dos vetores entrada a respostas e fetch das imagens

In [ ]:
X = np.empty((0,140500),int)
Y=np.empty((0,1),int)

print(X.shape)
print(Y.shape)
X,Y=loadXY("SegRe",X,Y)
print(X.shape)
print(Y.shape)

(0, 140500)
(0, 1)
(5760, 140500)
(5760, 1)


## Separação dos dados em conjunto de teste e treino

Aqui separamos nosso conjunto de dados em treino e teste. A proporção utilizado foi de 70% para treino e os 30% restantes para teste.

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, shuffle=True)
print(f"X_train dims={X_train.shape}")
print(f"Y_train dims={Y_train.shape}")
print(f"X_test dims={X_test.shape}")
print(f"Y_test dims={Y_test.shape}")

X_train dims=(4032, 140500)
Y_train dims=(4032, 1)
X_test dims=(1728, 140500)
Y_test dims=(1728, 1)


## Redução de dimensionalidade via decomposição em valores singulares

Aqui utilizamos o algoritmo de SVD para reduzir ainda mais a dimensionalidade dos nossos dados de forma a manter a variância deles conservadas. Reduzimos o número de features para 150 e conseguimos manter 86% da variância total com essas features. 

In [ ]:
n_comp = 150
svd = decomposition.TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd.fit(X_train.astype(np.float32))
print(f"Percentual da variancia explicada {svd.explained_variance_ratio_.sum()}")

train_features = svd.transform(X_train.astype(np.float32))
test_features = svd.transform(X_test.astype(np.float32))

Percentual da variancia explicada 0.8633547425270081


## Treino do classificador de vetores suporte 

Aqui fazemos um tuning para os hiperparâmetros do  modelo de SVM para classificação.| 

In [ ]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(train_features, Y_train.ravel())
print(clf.best_estimator_)

SVC(C=1000.0, class_weight='balanced', gamma=0.0001)


## Predição e relatorio para o SVC

Ao ajustarmos o modelo de SCV chegamos em uma métrica de acurácia de 0.23, o que é bem pequeno. Ao analisarmos melhor nossa matriz de confusão chegamos à conclusão de que o SVC em geral classifica todas as imagens como canecas e pratos, não conseguimos chegar a um motivo específico para esse comportamento.

In [ ]:
SVM_pred = clf.predict(test_features)
target_names=["Caneca","Chave","Escova","Dado","Meia","Fidget","Laranja","Garfo","Moeda","Prato"]
n_classes=len(target_names)
print("Relatorio para SVM")
print(classification_report(Y_test, SVM_pred, target_names=target_names))
print(confusion_matrix(Y_test, SVM_pred, labels=range(n_classes)))

Relatorio para SVM
              precision    recall  f1-score   support

      Caneca       0.13      0.91      0.23       179
       Chave       1.00      0.26      0.41       165
      Escova       1.00      0.06      0.11       168
        Dado       1.00      0.07      0.14       163
        Meia       1.00      0.09      0.16       176
      Fidget       0.95      0.12      0.22       170
     Laranja       1.00      0.13      0.23       179
       Garfo       1.00      0.02      0.04       175
       Moeda       1.00      0.22      0.36       174
       Prato       0.22      0.41      0.28       179

    accuracy                           0.23      1728
   macro avg       0.83      0.23      0.22      1728
weighted avg       0.82      0.23      0.22      1728

[[163   0   0   0   0   0   0   0   0  16]
 [ 95  43   0   0   0   1   0   0   0  26]
 [131   0  10   0   0   0   0   0   0  27]
 [112   0   0  12   0   0   0   0   0  39]
 [133   0   0   0  15   0   0   0   0  28]
 [134  

## Teste adiconal utilizando Multi Layer Perceptron (MLP)

Aqui ajustamos a rede neural MLP e, com a mesma métrica de acurácia, vemos que esse algoritmo obtém acurácia bem mais satisfatória de 0.73. Com a análise da matriz de confusão, vemos que o problema de classificações das canecas não acontece com esse algoritmo, apesar de que ainda conseguimos notar um número maior de classificações em pratos.

In [ ]:
MLP=neural_network.MLPClassifier(max_iter=1000)
MLP.fit(train_features, Y_train.ravel())
MLP_Score=MLP.score(test_features, Y_test)
MLP_Pred=MLP.predict(test_features)
print("Relatorio para MLP")
print(classification_report(Y_test, MLP_Pred, target_names=target_names))
print(confusion_matrix(Y_test, MLP_Pred, labels=range(n_classes)))

Relatorio para MLP
              precision    recall  f1-score   support

      Caneca       0.85      0.78      0.82       179
       Chave       0.85      0.73      0.78       165
      Escova       0.82      0.77      0.80       168
        Dado       0.76      0.60      0.67       163
        Meia       0.92      0.74      0.82       176
      Fidget       0.90      0.78      0.84       170
     Laranja       0.86      0.63      0.73       179
       Garfo       0.91      0.61      0.73       175
       Moeda       0.83      0.72      0.78       174
       Prato       0.39      0.97      0.55       179

    accuracy                           0.73      1728
   macro avg       0.81      0.73      0.75      1728
weighted avg       0.81      0.73      0.75      1728

[[140   3   2   5   5   4   0   0   2  18]
 [  3 120   1   9   0   3   1   1   1  26]
 [  1   0 130   1   1   0   0   4   1  30]
 [  3   5   4  97   2   1   1   0  11  39]
 [  5   5   0   1 130   3   1   1   2  28]
 [  6  

## Teste adiconal utilizando o classificador Naive Bayes

Ajustaremos por fim um modelo de Naive Bayes. Chegamos a uma acurácia de 0.33. E quanto a matriz de confusão, vemos que está possui a mesma característica de classificar muitos objetos em pratos.

In [ ]:
NB=naive_bayes.GaussianNB()
NB.fit(train_features, Y_train.ravel())
NB_Score=NB.score(test_features, Y_test)
NB_Pred=NB.predict(test_features)
print("Relatorio para Naive Bayes")
print(classification_report(Y_test, NB_Pred, target_names=target_names))
print(confusion_matrix(Y_test, NB_Pred, labels=range(n_classes)))

Relatorio para Naive Bayes
              precision    recall  f1-score   support

      Caneca       0.70      0.41      0.52       179
       Chave       0.49      0.27      0.35       165
      Escova       0.67      0.28      0.39       168
        Dado       0.42      0.05      0.09       163
        Meia       0.48      0.36      0.41       176
      Fidget       0.65      0.46      0.54       170
     Laranja       0.40      0.18      0.25       179
       Garfo       0.79      0.31      0.44       175
       Moeda       0.70      0.13      0.22       174
       Prato       0.15      0.83      0.25       179

    accuracy                           0.33      1728
   macro avg       0.54      0.33      0.35      1728
weighted avg       0.54      0.33      0.35      1728

[[ 74  22   0   4   7  22   4   0   0  46]
 [  3  44   0   0   5  10   7   0   4  92]
 [  4   1  47   0   7   0  10   4   2  93]
 [  4  12   0   8   4   0   3   0   1 131]
 [  9   4   2   0  63   1   4   0   0  93]

## Conclusões

Chegamos que o modelo melhor ajustado foi o de Multi Layer Perceptron (MLP), com uma acurácia de 0.73. 

Algumas hipóteses que tínhamos de que ,por exemplo, nosso algoritmo fosse confundir as moedas com os dados, pois ambos tem formatos e tamanhos similares não aconteceu. 

Porém, um problema que surgiu e pareceu constante nos três modelos ajustados foi o fato de que em geral, tende-se a classificar os objetos em pratos, ou seja, quase todas nossas classificações erradas foram classificadas como pratos. Não conseguimos chegar a um motivo específico para tal comportamento, mas vale ressaltar que as imagens dos pratos em geral eram as que ocupavam maior área da imagem. 